In [3]:
import sqlite3
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [4]:
con = sqlite3.connect("database.sqlite")

In [5]:
query = """
SELECT * 
FROM Match;
"""
match = pd.read_sql(sql=query, con=con)

match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [6]:
pd.set_option('display.max_columns', None)

In [28]:
query = """
SELECT 
    match_api_id, 
    league_id, 
    home_team_api_id, 
    away_team_api_id,  
    goal, shoton, 
    shotoff, 
    foulcommit,
    card, 
    cross,
    corner,
    possession
FROM Match;
"""
match_attributes = pd.read_sql(sql=query, con=con)

match_attributes.head()

,match_api_id,league_id,home_team_api_id,away_team_api_id,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,492473,1,9987,9993,None,None,None,None,None,None,None,None
1,492474,1,10000,9994,None,None,None,None,None,None,None,None
2,492475,1,9984,8635,None,None,None,None,None,None,None,None
3,492476,1,9991,9998,None,None,None,None,None,None,None,None
4,492477,1,7947,9985,None,None,None,None,None,None,None,None


In [29]:
match_attributes = match_attributes.dropna()

In [30]:
match_attributes.groupby("league_id").count()[
    ["goal", "shoton", "foulcommit", "card", "cross", "corner", "possession"]
]

,goal,shoton,foulcommit,card,cross,corner,possession
league_id,,,,,,,
1729,3040,3040,3040,3040,3040,3040,3040
4769,2026,2026,2026,2026,2026,2026,2026
7809,2448,2448,2448,2448,2448,2448,2448
10257,3014,3014,3014,3014,3014,3014,3014
13274,530,530,530,530,530,530,530
15722,7,7,7,7,7,7,7
19694,13,13,13,13,13,13,13
21518,3039,3039,3039,3039,3039,3039,3039
24558,100,100,100,100,100,100,100


In [10]:
match_attributes = match_attributes[
    (match_attributes["league_id"] == 1729)
    | (match_attributes["league_id"] == 4769)
    | (match_attributes["league_id"] == 7809)
    | (match_attributes["league_id"] == 10257)
     | (match_attributes["league_id"] == 21518)
].copy()

### Supplementary (cumulative) match info for home and away teams

In [11]:
def calculate_stats_both_teams(xml_document, home_team, away_team, card_type='y'):
    assert card_type == 'y' or card_type == 'r', "Please enter either y or r"
    tree = ET.fromstring(xml_document)
    stat_home_team = 0
    stat_away_team = 0
    
    if tree.tag == 'card':
        for child in tree.iter('value'):
            #Some xml docs have no card_type element in the tree. Comment section seems to have that information
            try:
                if child.find('comment').text == card_type:
                    if int(child.find('team').text) == home_team:
                        stat_home_team += 1
                    else:
                        stat_away_team += 1
            except AttributeError:
                #Some values in the xml doc don't have team values, so there isn't much we can do at this stage
                pass
                
        return stat_home_team, stat_away_team
    
    if tree.tag == 'possession':
        try:
            last_value = [child for child in tree.iter('value')][-1]
            return int(last_value.find('homepos').text), int(last_value.find('awaypos').text)
        except:
            return None, None
    
    for team in [int(stat.text) for stat in tree.findall('value/team')]:
        if team == home_team: 
            stat_home_team += 1
        else:
            stat_away_team += 1
    return stat_home_team, stat_away_team

#### On target shots

In [12]:
match_attributes[["on_target_shot_home_team", "on_target_shot_away_team"]] = match_attributes[
    ["goal", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["goal"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

#### Off target shots

In [13]:
match_attributes[["off_target_shot_home_team", "off_target_shot_away_team"]] = match_attributes[
    ["shotoff", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["shotoff"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

#### Fouls

In [14]:
match_attributes[["foul_home_team", "foul_away_team"]] = match_attributes[
    ["foulcommit", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["foulcommit"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

#### Yellow cards

In [15]:
match_attributes[["yellow_card_home_team", "yellow_card_away_team"]] = match_attributes[
    ["card", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["card"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

#### Red cards

In [16]:
match_attributes[["red_card_home_team", "red_card_away_team"]] = match_attributes[
    ["card", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["card"], x["home_team_api_id"], x["away_team_api_id"], card_type="r"
    ),
    axis=1,
    result_type="expand",
)

#### Crosses

In [17]:
match_attributes[["crosses_home_team", "crosses_away_team"]] = match_attributes[
    ["cross", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["cross"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

#### Corners

In [18]:
match_attributes[["corner_home_team", "corner_away_team"]] = match_attributes[
    ["corner", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["corner"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

#### Possession

In [19]:
match_attributes[["possession_home_team", "possession_away_team"]] = match_attributes[
    ["possession", "home_team_api_id", "away_team_api_id"]
].apply(
    lambda x: calculate_stats_both_teams(
        x["possession"], x["home_team_api_id"], x["away_team_api_id"]
    ),
    axis=1,
    result_type="expand",
)

In [20]:
match_attr = match_attributes.drop(columns=['league_id', 'home_team_api_id', 'away_team_api_id', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession'])

In [21]:
match_attr.head()

,match_api_id,on_target_shot_home_team,on_target_shot_away_team,off_target_shot_home_team,off_target_shot_away_team,foul_home_team,foul_away_team,yellow_card_home_team,yellow_card_away_team,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team
1728,489042,1,1,10,9,16,11,3,0,0,0,24,9,6,6,55.0,45.0
1729,489043,1,0,13,3,11,9,0,0,0,0,21,7,7,5,66.0,34.0
1730,489044,0,1,3,5,13,12,0,2,0,0,15,19,1,8,46.0,54.0
1731,489045,2,1,7,15,14,13,2,1,0,0,15,27,6,10,52.0,48.0
1732,489046,4,2,4,5,11,13,0,1,0,0,16,16,7,8,52.0,48.0


### Goals by player per match

In [22]:
def extract_goals(xml_document, match_id):
    tree = ET.fromstring(xml_document)
    res = {}

    for stat in tree.findall("value"):
        # some stats in the goal xml is related with penalties - need to filter out only goal related information
        if not stat.findall("stats/goals"):
            continue
        player = stat.find("player1")
        team = stat.find("team")

        if player is not None:
            player = player.text
            team = team.text
            if player in res:
                res[player]["goals"] = res[player]["goals"] + 1
            else:
                res[player] = {"goals": 1, "team_id": team}
    return match_id, res


goals_by_player = pd.DataFrame(columns=["match_id", "player_id", "goals", "team_id"])

for index, row in match_attributes.iterrows():
    match_id, scorers = extract_goals(row["goal"], row["match_api_id"])
    for scorer in scorers:
        goals_by_player = goals_by_player.append(
            {
                "match_id": match_id,
                "player_id": scorer,
                "goals": scorers[scorer]["goals"],
                "team_id": scorers[scorer]["team_id"],
            },
            ignore_index=True,
        )

In [23]:
goals_by_player.head()

,match_id,player_id,goals,team_id
0,489042,37799,1,10261
1,489042,24148,1,10260
2,489043,26181,1,9825
3,489044,30853,1,8650
4,489045,23139,2,8654
